# Matrix Factorization

In [78]:
import pandas as pd 
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from scipy import sparse 

class MF(object):
    """docstring for CF"""
    def __init__(self, Y_data, K, lam = 0.1, Xinit = None, Winit = None, 
                 learning_rate = 0.5, max_iter = 1000, print_every = 100, user_based = 0):
        self.Y_raw = Y_data.copy()
        self.Y_data = Y_data.copy()
        self.K = K
        self.lam = lam
        self.learning_rate = learning_rate
        self.max_iter = max_iter
        self.print_every = print_every
        self.user_based = user_based
        # number of users and items. Remember to add 1 since id starts from 0
        self.n_users = int(np.max(Y_data[:, 0])) + 1 
        self.n_items = int(np.max(Y_data[:, 1])) + 1
        
        if Xinit is None: 
            self.X = np.random.randn(self.n_items, K)
        else:
            self.X = Xinit 
        
        if Winit is None: 
            self.W = np.random.randn(K, self.n_users)
        else: 
            self.W = Winit
        
        # item biases
        self.b = np.random.randn(self.n_items)
        self.d = np.random.randn(self.n_users)
        #self.all_users = self.Y_data[:,0] # all users (may be duplicated)
        self.n_ratings = Y_data.shape[0]
#         self.mu = np.mean(Y_data[:, 2])
        self.mu = 0
 

    def normalize_Y(self):
        if self.user_based:
            user_col = 0
            item_col = 1
            n_objects = self.n_users
        else:
            user_col = 1
            item_col = 0 
            n_objects = self.n_items
        self.muu = np.zeros(n_objects)
        self.Y_data = self.Y_raw
        return
    
        if self.user_based:
            user_col = 0
            item_col = 1
            n_objects = self.n_users
        else:
            user_col = 1
            item_col = 0 
            n_objects = self.n_items

        users = self.Y_data[:, user_col] 
        self.muu = np.zeros((n_objects,))
        for n in xrange(n_objects):
            # row indices of rating done by user n
            # since indices need to be integers, we need to convert
            ids = np.where(users == n)[0].astype(np.int32)
            # indices of all ratings associated with user n
            item_ids = self.Y_data[ids, item_col] 
            # and the corresponding ratings 
            ratings = self.Y_data[ids, 2]
            # take mean
            m = np.mean(ratings) 
#             print m
            if np.isnan(m):
                m = 0 # to avoid empty array and nan value
            self.muu[n] = m
            # normalize
            self.Y_data[ids, 2] = ratings - m
            
            
    def loss(self):
        L = 0 
        for i in xrange(self.n_ratings):
            # user, item, rating
            n, m, rate = int(self.Y_data[i, 0]), int(self.Y_data[i, 1]), self.Y_data[i, 2]
            L += 0.5*(self.X[m, :].dot(self.W[:, n]) + self.b[m] + self.d[n] + self.mu - rate)**2
            
        # regularization, don't ever forget this 
        L /= self.n_ratings
        L += 0.5*self.lam*(np.sum(self.X**2) + np.sum(self.W**2))
#         L += 0.5*self.lam*(np.linalg.norm(self.X, 'fro')**2 + np.linalg.norm(self.W, 'fro')**2 + \
#                           np.linalg.norm(self.b) + np.linalg.norm(self.d))
        return L 

    
    def get_items_rated_by_user(self, user_id):
        """
        get all items which are rated by user n, and the corresponding ratings
        """
        # y = self.Y_data_n[:,0] # all users (may be duplicated)
        # item indices rated by user_id
        # we need to +1 to user_id since in the rate_matrix, id starts from 1 
        # while index in python starts from 0
        ids = np.where(self.Y_data[:,0] == user_id)[0] 
        item_ids = self.Y_data[ids, 1].astype(np.int32) # index starts from 0 
        ratings = self.Y_data[ids, 2]
        return (item_ids, ratings)
        
        
    def get_users_who_rate_item(self, item_id):
        """
        get all users who rated item m and get the corresponding ratings
        """
        ids = np.where(self.Y_data[:,1] == item_id)[0] 
        user_ids = self.Y_data[ids, 0].astype(np.int32)
        ratings = self.Y_data[ids, 2]
        return (user_ids, ratings)
        
    def updateX(self): # and b 
#         for i in range(10):
            for m in xrange(self.n_items):
                user_ids, ratings = self.get_users_who_rate_item(m)

                Wm = self.W[:, user_ids]
                dm = self.d[user_ids]
                for i in range(30):
                    xm = self.X[m, :]

                    error = xm.dot(Wm) + self.b[m] + dm + self.mu - ratings 

                    grad_xm = error.dot(Wm.T)/self.n_ratings + self.lam*xm
                    grad_bm = np.sum(error)/self.n_ratings# + self.lam*self.b[m]
                    self.X[m, :] -= self.learning_rate*grad_xm.reshape(-1)
                    self.b[m]    -= self.learning_rate*grad_bm
    
    def updateW(self): # and d 
#         for i in range(10):
            for n in xrange(self.n_users):
                item_ids, ratings = self.get_items_rated_by_user(n)
                Xn = self.X[item_ids, :]
                bn = self.b[item_ids]
                for i in range(30):
                    wn = self.W[:, n]
                    error = Xn.dot(wn) + bn + self.mu + self.d[n] - ratings
                    grad_wn = Xn.T.dot(error)/self.n_ratings + self.lam*wn
                    grad_dn = np.sum(error)/self.n_ratings# + self.lam*self.d[n]
                    self.W[:, n] -= self.learning_rate*grad_wn.reshape(-1)
                    self.d[n]    -= self.learning_rate*grad_dn
    
    def fit(self):
        self.normalize_Y()
        for it in xrange(self.max_iter):
            self.updateW()
            self.updateX()
            if (it + 1) % self.print_every == 0:
                rmse_train = self.evaluate_RMSE(self.Y_raw)
                print 'iter =', it + 1, ', loss =', self.loss(), ', RMSE train =', rmse_train
    
    
    def pred(self, u, i):
        """ 
        predict the rating of user u for item i 
        if you need the un
        """
        u = int(u)
        i = int(i)
        if self.user_based == 1:
            bias = self.muu[u]
        else:
            bias = self.muu[i]
        
        pred = self.X[i, :].dot(self.W[:, u]) + self.b[i] + self.d[u] + bias
        return max(0, min(5, pred))
        
    
    def pred_for_user(self, user_id):
        ids = np.where(self.Y_data_n[:, 0] == user_id)[0]
        items_rated_by_u = self.Y_data_n[ids, 1].tolist()              
        
        y_pred = self.X.dot(self.W[:, user_id])
        predicted_ratings= []
        for i in xrange(self.n_items):
            if i not in items_rated_by_u:
                predicted_ratings.append((i, y_pred[i]))
        
        return predicted_ratings
    
    def evaluate_RMSE(self, rate_test):
        n_tests = rate_test.shape[0]
        SE = 0 # squared error
        for n in xrange(n_tests):
            pred = self.pred(rate_test[n, 0], rate_test[n, 1])
#             print pred, rate_test[n, 2]
            SE += (pred - rate_test[n, 2])**2 

        RMSE = np.sqrt(SE/n_tests)
        return RMSE
        
r_cols = ['user_id', 'item_id', 'rating']
ratings = pd.read_csv('ex.dat', sep = ' ', names = r_cols, encoding='latin-1')
Y_data = ratings.as_matrix()


rs = MF(Y_data, K = 3, max_iter = 1000, print_every = 100, lam = 0.1)

rs.fit()
rs.pred(6, 1)
print rs.evaluate_RMSE(Y_data)

iter = 100 , loss = 0.677157375821 , RMSE train = 0.64593932342
iter = 200 , loss = 0.677157375821 , RMSE train = 0.64593932342
iter = 300 , loss = 0.677157375821 , RMSE train = 0.64593932342
iter = 400 , loss = 0.677157375821 , RMSE train = 0.64593932342
iter = 500 , loss = 0.677157375821 , RMSE train = 0.64593932342
iter = 600 , loss = 0.677157375821 , RMSE train = 0.64593932342
iter = 700 , loss = 0.677157375821 , RMSE train = 0.64593932342
iter = 800 , loss = 0.677157375821 , RMSE train = 0.64593932342
iter = 900 , loss = 0.677157375821 , RMSE train = 0.64593932342
iter = 1000 , loss = 0.677157375821 , RMSE train = 0.64593932342
0.64593932342


# Áp dụng lên MovieLens 100k

In [79]:
r_cols = ['user_id', 'movie_id', 'rating', 'unix_timestamp']

ratings_base = pd.read_csv('ml-100k/ua.base', sep='\t', names=r_cols, encoding='latin-1')
ratings_test = pd.read_csv('ml-100k/ua.test', sep='\t', names=r_cols, encoding='latin-1')

rate_train = ratings_base.as_matrix()
rate_test = ratings_test.as_matrix()

# indices start from 0
rate_train[:, :2] -= 1
rate_test[:, :2] -= 1

In [80]:
rs = MF(rate_train, K = 50, lam = .01, print_every = 1, learning_rate = 50, max_iter = 30, user_based = 0)
rs.fit()
# evaluate on test data
RMSE = rs.evaluate_RMSE(rate_test)
print '\nUser-based MF, RMSE =', RMSE

iter = 1 , loss = 0.838481001359 , RMSE train = 1.23999082333
iter = 2 , loss = 0.553435939663 , RMSE train = 1.05073980923
iter = 3 , loss = 0.489474924517 , RMSE train = 0.98880920639
iter = 4 , loss = 0.460787309927 , RMSE train = 0.959589143689
iter = 5 , loss = 0.445323583187 , RMSE train = 0.943443311241
iter = 6 , loss = 0.436098313928 , RMSE train = 0.933660017695
iter = 7 , loss = 0.430232948009 , RMSE train = 0.927384199064
iter = 8 , loss = 0.426335046355 , RMSE train = 0.923187150012
iter = 9 , loss = 0.423655923684 , RMSE train = 0.920292673307
iter = 10 , loss = 0.421762831228 , RMSE train = 0.918242322442
iter = 11 , loss = 0.420392461297 , RMSE train = 0.916753616572
iter = 12 , loss = 0.419378406058 , RMSE train = 0.915648686117
iter = 13 , loss = 0.418612373079 , RMSE train = 0.914809614153
iter = 14 , loss = 0.418022212273 , RMSE train = 0.914161256614
iter = 15 , loss = 0.417558900732 , RMSE train = 0.913651258427
iter = 16 , loss = 0.41718855971 , RMSE train = 0.91

In [62]:
rs.d

array([ -5.75313645e-02,  -5.58802976e-02,   4.37430623e-01,
         1.27722083e-01,   3.35361257e-01,  -2.36411764e-01,
         2.20418177e-01,   1.10353770e-01,  -5.18767230e-01,
        -2.99089254e-01,  -7.72862725e-02,  -3.53361838e-01,
        -2.99119442e-01,  -5.58966493e-01,   3.60940768e-01,
        -4.10568719e-01,  -5.79402842e-01,   5.27044850e-02,
         6.12365727e-02,  -3.32379174e-01,   3.89522087e-01,
        -1.56492332e-01,  -1.14239557e-01,  -3.55299201e-01,
        -4.29373530e-01,   2.43997074e-01,  -1.60826983e-01,
         2.23661494e-02,   4.50742502e-02,   1.25668897e-01,
         2.44312686e-02,  -1.68173365e-01,   5.34576784e-01,
         9.25468683e-03,   4.98794852e-01,   2.79010301e-01,
         5.02544084e-02,   2.74386727e-01,   3.23768725e-02,
         2.19885211e-02,  -1.79003217e-01,   1.91057231e-02,
         8.75344491e-03,  -1.44753477e-01,   1.94551264e-01,
        -3.54697661e-01,   3.08756288e-01,  -1.07651288e-01,
         1.98988177e-01,

# MovieLens 1M

In [ ]:
r_cols = ['user_id', 'movie_id', 'rating', 'unix_timestamp']

ratings_base = pd.read_csv('ml-1m/ratings.dat', sep='::', names=r_cols, encoding='latin-1')


In [ ]:
rs = MF(rate_train, K = 2, lam = 0.1, print_every = 2, learning_rate = 2, 
        max_iter = 50, user_based = 0, Xinit = rs.X, Winit = rs.W)
rs.fit()
# evaluate on test data
RMSE = rs.evaluate_RMSE(rate_test)
print '\nItem-based MF, RMSE =', RMSE